In [1]:
from google.colab import drive 
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
import os
os.chdir("drive/MyDrive/Cognizant_final_hackathon/topic_7")

FileNotFoundError: ignored

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
def get_simple_pos(tag):
  if tag.startswith('J'):
    return wordnet.ADJ
  elif tag.startswith('V'):
    return wordnet.VERB
  elif tag.startswith('N'):
    return wordnet.NOUN
  elif tag.startswith('R'):
    return wordnet.ADV
  else:
    return wordnet.NOUN

In [ ]:
def to_nltk_format(words):
  final_doc = []
  for index,row in words.iterrows():
    output_list = []
    output_words = []
    words =row['tweet']
    words = words.split()
    for w in words:
      if w.isalpha():
        if w not in stops and len(w)>2:
          output_words.append(w)
    output_list = [output_words] + [row['class']]
    final_doc.append(output_list)
  return final_doc

In [ ]:
def to_prediction_format(words):
  final_doc = []
  for index,row in words.iterrows():
    output_list = []
    output_words = []
    words =row['tweet']
    words = words.split()
    for w in words:
      if w.isalpha():
        if w not in stops and len(w)>2:
          output_words.append(w)
    output_list = output_words
    final_doc.append(output_list)
  return final_doc

In [ ]:
def reg_fun(df_):
  # to list
  tweets = df_.tweet.values
  #to lower
  tweets = [i.lower() for i in tweets]
  # removing handles with @
  tweets_no_handle = [  re.sub("&*@\w+",'',sent) for sent in tweets]
  # removing hashtag
  tweets_clean = [re.sub("#\w+",'',tweet) for tweet in tweets_no_handle]
  # remove urls: 
  tweets_clean = [re.sub("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|''[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",'',tweet) for tweet in tweets_clean]
  # remove rt
  #tweets_clean = [re.sub("[(rt :)]",' ',tweet) for tweet in tweets_clean]
  #remove &;
  tweets_clean = [re.sub(":*;*&;*",'',tweet) for tweet in tweets_clean]
  #remove &;
  tweets_clean = [re.sub("[...\"\;\']",'',tweet) for tweet in tweets_clean]
  # to dataframe
  temp = pd.DataFrame(data = tweets_clean, columns=['tweet'])
  df_2 = df_.copy()
  df_2['tweet'] = temp['tweet'] 
  return df_2

In [ ]:
stops = set(stopwords.words('english'))
punctuations =list(string.punctuation)
extra = ['&;',  ':',  'a', "a", 'i', 'rt' , '&;:',"'" , 'if', 'but', 'rt', 'just', 'you','to' , '&;&;', '":'  ,  '.' , '&;&;&;' , ',' , '-'  , 'n', '"' , '...'  , '!' , '2' , '?', '..' , "'" , '&;&;&;&;&;' , '"i' , '|' , '3' , '1' ,'@', '4', "'"]
#stops = stops.append(extra)
stops.update(punctuations)
stops.update(extra) 
#stops.append()
stops.remove('not')
stops

In [ ]:
def rm_train_stopwords(df_):
  dataframe = pd.DataFrame(columns = ['tweet','class'])
  for index,row in df_.iterrows():
    clean_sentence=[]
    tokens = word_tokenize(row['tweet'])      
    for w in tokens:
        w = w.lower()
        if w.isalpha() and len(w)>2:
          if w not in stops:
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1])) 
            clean_sentence.append(clean_word)     
    dataframe = dataframe.append({'class':row['class'], 'tweet': ' '.join(clean_sentence)}, ignore_index = True)
  return dataframe

In [ ]:
def rm_test_stopwords(df_):
  dataframe = pd.DataFrame(columns = ['tweet'])
  for index,row in df_.iterrows():
    clean_sentence=[]
    tokens = word_tokenize(row['tweet'])      
    for w in tokens:
      w = w.lower()
      if w.isalpha() and len(w)>2:
        if w not in stops:
          pos = pos_tag([w])
          clean_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1])) 
          clean_sentence.append(clean_word)     
    dataframe = dataframe.append({'tweet': ' '.join(clean_sentence)}, ignore_index = True)
  return dataframe

In [ ]:
def get_feature_dict(words):
  current_features = {}
  words_set = set(words)
  for w in features:
    current_features[w] = w in words_set
  return current_features

In [ ]:
data = pd.read_csv("hate_speech_train.csv")
test_data = pd.read_csv("personal_twitter_data.csv")
raw_test = pd.read_excel("hate_speech_test.xlsx")

In [ ]:
data.head()

,count,hate_speech,offensive_language,neither,class,tweet
0,3,1,2,0,1,RT @HENNERGIZED: &#128557;&#128557;&#128557;&#...
1,3,0,3,0,1,We are back bitches! @vnpacheco21 @xoxoclaire_...
2,3,0,3,0,1,"RT @SuperrrrMcNasty: Lmfao , this bitch was gi..."
3,3,0,3,0,1,@bradley_eckman don't say shit like that lil n...
4,3,1,2,0,1,Young jezzy dat nigguh


In [ ]:
test_data.head()

,tweet,class
0,"RT @RedTvvitter: ""Does my Mewtwo make your pus...",1
1,@kieffer_jason and who said I didn't like myse...,1
2,Get this bitch off my TV bruh,1
3,Lmfao bitch got her shit rocked,1
4,Verizon customer service is trash as fuck. I'v...,1


In [ ]:
raw_test.head()

,class,tweet
0,NaN,"RT @RedTvvitter: ""Does my Mewtwo make your pus..."
1,NaN,@kieffer_jason and who said I didn't like myse...
2,NaN,Get this bitch off my TV bruh
3,NaN,Lmfao bitch got her shit rocked
4,NaN,Verizon customer service is trash as fuck. I'v...


In [ ]:
print("data: {}".format(data.shape))
print("labelled: {}".format(test_data.shape))
print("test: {}".format(raw_test.shape))

data: (19826, 6)
labelled: (4957, 2)
test: (4957, 2)


In [ ]:
print('train:{}'.format(data.isnull().sum()))

train:count                 0
hate_speech           0
offensive_language    0
neither               0
class                 0
tweet                 0
dtype: int64


In [ ]:
print('git:{}'.format(test_data.isnull().sum()))

git:tweet    0
class    0
dtype: int64


In [ ]:
print('test:{}'.format(raw_test.isnull().sum()))

test:class    4957
tweet       0
dtype: int64


In [ ]:
test = raw_test.drop('class', axis=1)
#test = raw_test

In [ ]:
test.head()

,tweet
0,"RT @RedTvvitter: ""Does my Mewtwo make your pus..."
1,@kieffer_jason and who said I didn't like myse...
2,Get this bitch off my TV bruh
3,Lmfao bitch got her shit rocked
4,Verizon customer service is trash as fuck. I'v...


In [ ]:
df = pd.DataFrame(data = data[['tweet','class']], columns = ['tweet','class'])

In [ ]:
df.head()

,tweet,class
0,RT @HENNERGIZED: &#128557;&#128557;&#128557;&#...,1
1,We are back bitches! @vnpacheco21 @xoxoclaire_...,1
2,"RT @SuperrrrMcNasty: Lmfao , this bitch was gi...",1
3,@bradley_eckman don't say shit like that lil n...,1
4,Young jezzy dat nigguh,1


In [ ]:
df.tweet.values

array(['RT @HENNERGIZED: &#128557;&#128557;&#128557;&#8220;@StopBeingSober: Yall say "Why you dating lil girls" like mature hoes just on a rampage outside.&#8221;',
       'We are back bitches! @vnpacheco21 @xoxoclaire_ @treyhunter_ @Medgmon @delaney_jade @sean_steezy @alexistiefa',
       'RT @SuperrrrMcNasty: Lmfao , this bitch was giving head in the back of a classroom she better go tf ahead !',
       ...,
       'RT @FunnyOGtweets: When rednecks fool around with the aux cord http://t.co/Tyzuro7qR5',
       "@PaulMoon8 @yurista4life @justinmiculka01 you fuckin Yankees I'll be there in 2 months I'll meet you pussy",
       "RT @liyyy_: yall females really be cool with cuffin hoe ass niggas &#128514; nope. couldn't be me."],
      dtype=object)

In [ ]:
df_2 = reg_fun(df)
df_t = reg_fun(test_data)
df_test = reg_fun(test)
df_2.head()

,tweet,class
0,rt : : yall say why you dating lil girls like ...,1
1,we are back bitches!,1
2,"rt : lmfao , this bitch was giving head in the...",1
3,dont say shit like that lil nig or ill give ...,1
4,young jezzy dat nigguh,1


In [ ]:
df_t.head()

,tweet,class
0,rt : does my mewtwo make your pussy wet?,1
1,and who said i didnt like myself where the fu...,1
2,get this bitch off my tv bruh,1
3,lmfao bitch got her shit rocked,1
4,verizon customer service is trash as fuck ive ...,1


In [ ]:
df_test.head()

,tweet
0,rt : does my mewtwo make your pussy wet?
1,and who said i didnt like myself where the fu...
2,get this bitch off my tv bruh
3,lmfao bitch got her shit rocked
4,verizon customer service is trash as fuck ive ...


In [ ]:
#dataframe = rm_train_stopwords(df_2)
#dataframe_t = rm_train_stopwords(df_t)
#dataframe_test = rm_test_stopwords(df_test)

dataframe = df_2
dataframe_t = df_t
dataframe_test = df_test

In [ ]:
dataframe.head(10)

,tweet,class
0,rt : : yall say why you dating lil girls like ...,1
1,we are back bitches!,1
2,"rt : lmfao , this bitch was giving head in the...",1
3,dont say shit like that lil nig or ill give ...,1
4,young jezzy dat nigguh,1
5,same shit rt : but your bm tho? : if my ex wa...,1
6,rt : so cute! :) rt : shy bunny,2
7,this bitch always got shit to say about what o...,1
8,rt : i just wanna pimp these hoe$ amp ride rarris,1
9,i heard them same pussy niggas hatin !,1


In [ ]:
dataframe_t.head()

,tweet,class
0,rt : does my mewtwo make your pussy wet?,1
1,and who said i didnt like myself where the fu...,1
2,get this bitch off my tv bruh,1
3,lmfao bitch got her shit rocked,1
4,verizon customer service is trash as fuck ive ...,1


In [ ]:
dataframe_test.head()

,tweet
0,rt : does my mewtwo make your pussy wet?
1,and who said i didnt like myself where the fu...
2,get this bitch off my tv bruh
3,lmfao bitch got her shit rocked
4,verizon customer service is trash as fuck ive ...


In [ ]:
def basic_clean(text):
  """
  Function to clean up the data. All the words that
  are not designated as a stop word is then lemmatized after
  encoding and basic regex parsing are performed.
  """
  words = text.split()
  return [word for word in words ]

In [ ]:
#words = basic_clean(str(dataframe['tweet'].tolist()))

In [ ]:
#(pd.Series(nltk.ngrams(words, 3)).value_counts())[:10]

('hoe, aint, loyal',)      28
(bitch, as, nigga)         28
(bitch, look, like)        23
(stupid, as, bitch)        17
(as, bitch, aint)          17
(park, slope, top)         16
(slope, top, rat)          16
(top, rat, spa',)          16
('best, asian, massage)    16
(bitch, thing, like',)     15
dtype: int64

In [ ]:
#(pd.Series(nltk.ngrams(words, 2)).value_counts())[:100]

(look, like)     194
(bitch, get)     176
(bad, bitch)     152
(like, bitch)    144
(bitch, as)      134
                ... 
(talk, bout)      31
(bitch, try)      31
(fuck, nigga)     30
(side, bitch)     30
(bitch, gon)      30
Length: 100, dtype: int64

In [ ]:
#flag = (look, like) in 'hey look like'

NameError: ignored

In [ ]:
documents = to_nltk_format(dataframe)
documents_t = to_nltk_format(dataframe_t)
documents_test = to_prediction_format(dataframe_test)

In [ ]:
documents

[[['yall',
   'say',
   'dating',
   'lil',
   'girls',
   'like',
   'mature',
   'hoes',
   'rampage',
   'outside'],
  1],
 [['back'], 1],
 [['lmfao', 'bitch', 'giving', 'head', 'back', 'classroom', 'better', 'ahead'],
  1],
 [['dont', 'say', 'shit', 'like', 'lil', 'nig', 'ill', 'give', 'stamp'], 1],
 [['young', 'jezzy', 'dat', 'nigguh'], 1],
 [['shit', 'wanna', 'fuck', 'may', 'god', 'aint', 'hoe'], 1],
 [['shy', 'bunny'], 2],
 [['bitch', 'always', 'got', 'shit', 'say', 'people', 'like'], 1],
 [['wanna', 'pimp', 'amp', 'ride', 'rarris'], 1],
 [['heard', 'pussy', 'niggas', 'hatin'], 1],
 [['inmate', 'sums', 'return', 'means'], 2],
 [['want', 'ian', 'trippin', 'hoes'], 1],
 [['yall',
   'men',
   'need',
   'watch',
   'cream',
   'pies',
   'bitches',
   'yeast',
   'infections',
   'amp',
   'day',
   'monistat',
   'treatment',
   'know',
   'difference'],
  1],
 [['gradeschool',
   'dance',
   'tried',
   'worm',
   'impress',
   'bitch',
   'slammed',
   'dick',
   'unforgiving',

In [ ]:
documents

[[['yall',
   'say',
   'dating',
   'lil',
   'girls',
   'like',
   'mature',
   'hoes',
   'rampage',
   'outside'],
  1],
 [['back'], 1],
 [['lmfao', 'bitch', 'giving', 'head', 'back', 'classroom', 'better', 'ahead'],
  1],
 [['dont', 'say', 'shit', 'like', 'lil', 'nig', 'ill', 'give', 'stamp'], 1],
 [['young', 'jezzy', 'dat', 'nigguh'], 1],
 [['shit', 'wanna', 'fuck', 'may', 'god', 'aint', 'hoe'], 1],
 [['shy', 'bunny'], 2],
 [['bitch', 'always', 'got', 'shit', 'say', 'people', 'like'], 1],
 [['wanna', 'pimp', 'amp', 'ride', 'rarris'], 1],
 [['heard', 'pussy', 'niggas', 'hatin'], 1],
 [['inmate', 'sums', 'return', 'means'], 2],
 [['want', 'ian', 'trippin', 'hoes'], 1],
 [['yall',
   'men',
   'need',
   'watch',
   'cream',
   'pies',
   'bitches',
   'yeast',
   'infections',
   'amp',
   'day',
   'monistat',
   'treatment',
   'know',
   'difference'],
  1],
 [['gradeschool',
   'dance',
   'tried',
   'worm',
   'impress',
   'bitch',
   'slammed',
   'dick',
   'unforgiving',

In [ ]:
documents_t

[[['mewtwo', 'make', 'pussy'], 1],
 [['said',
   'didnt',
   'like',
   'fuck',
   'got',
   'shit',
   'made',
   'stfu',
   'stop',
   'assuming',
   'bitch',
   'not',
   'ugly'],
  1],
 [['get', 'bitch', 'bruh'], 1],
 [['lmfao', 'bitch', 'got', 'shit', 'rocked'], 1],
 [['verizon',
   'customer',
   'service',
   'trash',
   'fuck',
   'ive',
   'store',
   'havent',
   'greeted',
   'nothing'],
  1],
 [['youve', 'gone', 'broke', 'wrong', 'heart', 'drove', 'redneck', 'crazy'],
  2],
 [['bitch', 'ass', 'arod', 'shorty', 'dont', 'hit', 'back'], 1],
 [['clock',
   'hits',
   'oclock',
   'imma',
   'need',
   'amp',
   'brightest',
   'hoes',
   'door',
   'give'],
  1],
 [['long',
   'beach',
   'california',
   'people',
   'automatically',
   'think',
   'ima',
   'bitch',
   'hahha'],
  1],
 [['wicked', 'witch', 'east', 'send', 'vile', 'bitch', 'syria'], 1],
 [['aint', 'squad', 'lil', 'bitch'], 1],
 [['thought', 'would', 'think', 'cuteyou', 'like', 'shut', 'clam'], 2],
 [['bitch',


In [ ]:
documents_test

[['mewtwo', 'make', 'pussy'],
 ['said',
  'didnt',
  'like',
  'fuck',
  'got',
  'shit',
  'made',
  'stfu',
  'stop',
  'assuming',
  'bitch',
  'not',
  'ugly'],
 ['get', 'bitch', 'bruh'],
 ['lmfao', 'bitch', 'got', 'shit', 'rocked'],
 ['verizon',
  'customer',
  'service',
  'trash',
  'fuck',
  'ive',
  'store',
  'havent',
  'greeted',
  'nothing'],
 ['youve', 'gone', 'broke', 'wrong', 'heart', 'drove', 'redneck', 'crazy'],
 ['bitch', 'ass', 'arod', 'shorty', 'dont', 'hit', 'back'],
 ['clock',
  'hits',
  'oclock',
  'imma',
  'need',
  'amp',
  'brightest',
  'hoes',
  'door',
  'give'],
 ['long',
  'beach',
  'california',
  'people',
  'automatically',
  'think',
  'ima',
  'bitch',
  'hahha'],
 ['wicked', 'witch', 'east', 'send', 'vile', 'bitch', 'syria'],
 ['aint', 'squad', 'lil', 'bitch'],
 ['thought', 'would', 'think', 'cuteyou', 'like', 'shut', 'clam'],
 ['bitch',
  'wit',
  'baby',
  'ask',
  'switch',
  'seats',
  'family',
  'could',
  'together',
  'said',
  'bitch',


In [ ]:
#training_documents = documents[0:14869]
#testing_documents = documents[14869:]
training_documents = documents
testing_documents = documents_test
testing_documents_t = documents_t

In [ ]:
all_words = []
for doc in training_documents:
    all_words += doc[0]
freq = nltk.FreqDist(all_words)
common = freq.most_common(1000)
features = [i[0] for i in common]

In [ ]:
all_words

In [ ]:
&;,  :,  a, i, rt , &;: , if, but, just, to , &;&;, ":  ,  dot(.) , &;&;&; , comma(,) , -  , n, " , ...  , ! , 2 , ?, .. , ' , &;&;&;&;&; , "i , | , 3 , 1 ,@ , 4, 

In [ ]:
common

[('bitch', 6126),
 ('bitches', 2353),
 ('like', 2165),
 ('hoes', 1772),
 ('pussy', 1599),
 ('dont', 1424),
 ('hoe', 1401),
 ('ass', 1196),
 ('get', 1146),
 ('fuck', 1111),
 ('got', 1036),
 ('not', 928),
 ('shit', 927),
 ('aint', 921),
 ('nigga', 901),
 ('trash', 854),
 ('lol', 831),
 ('amp', 651),
 ('know', 638),
 ('niggas', 613),
 ('love', 558),
 ('one', 546),
 ('cant', 534),
 ('yall', 529),
 ('fucking', 512),
 ('thats', 483),
 ('youre', 456),
 ('want', 448),
 ('bad', 437),
 ('good', 402),
 ('make', 388),
 ('man', 384),
 ('still', 383),
 ('say', 380),
 ('see', 371),
 ('hate', 366),
 ('look', 365),
 ('need', 364),
 ('never', 358),
 ('think', 357),
 ('time', 347),
 ('back', 341),
 ('real', 332),
 ('really', 327),
 ('people', 318),
 ('even', 316),
 ('girl', 307),
 ('white', 306),
 ('faggot', 298),
 ('would', 297),
 ('lmao', 285),
 ('said', 282),
 ('right', 280),
 ('let', 276),
 ('bird', 273),
 ('wanna', 270),
 ('bout', 269),
 ('wit', 268),
 ('damn', 261),
 ('little', 259),
 ('dick', 257)

In [ ]:
def get_feature_dict(words):
  current_features = {}
  words_set = set(words)
  for w in features:
    current_features[w] = w in words_set
  return current_features

In [ ]:
output = get_feature_dict(training_documents[0][0])
output

In [ ]:
training_data = [(get_feature_dict(doc), category) for doc, category in training_documents]
testing_data = [(get_feature_dict(doc)) for doc in testing_documents]
testing_data_t = [(get_feature_dict(doc), category) for doc, category in testing_documents_t]

In [ ]:
from nltk import NaiveBayesClassifier
classfier = NaiveBayesClassifier.train(training_data)
nltk.classify.accuracy(classfier, testing_data_t)

0.8721000605204761

In [ ]:
nltk.classify.accuracy(classfier, testing_data_t)

0.8811157066478362

In [ ]:
tests=['I really like it', 
'I do not think this is good one', 
'this is good one',
'I hate the show!',
'Get this bitch off my TV bruh',
'Perhaps not diehard Yankee fan no point watch debacle LOL',
'Fuck good',
'love asian bitches']

for test in tests:
  t_features = {word: (word in word_tokenize(test.lower())) for word in features}
  print(t_features," : ", classfier.classify(t_features))
  #print(t_features)

{'bitch': False, 'bitches': False, 'like': True, 'hoes': False, 'pussy': False, 'dont': False, 'hoe': False, 'ass': False, 'get': False, 'fuck': False, 'got': False, 'not': False, 'shit': False, 'aint': False, 'nigga': False, 'trash': False, 'lol': False, 'amp': False, 'know': False, 'niggas': False, 'love': False, 'one': False, 'cant': False, 'yall': False, 'fucking': False, 'thats': False, 'youre': False, 'want': False, 'bad': False, 'good': False, 'make': False, 'man': False, 'still': False, 'say': False, 'see': False, 'hate': False, 'look': False, 'need': False, 'never': False, 'think': False, 'time': False, 'back': False, 'real': False, 'really': True, 'people': False, 'even': False, 'girl': False, 'white': False, 'faggot': False, 'would': False, 'lmao': False, 'said': False, 'right': False, 'let': False, 'bird': False, 'wanna': False, 'bout': False, 'wit': False, 'damn': False, 'little': False, 'dick': False, 'stop': False, 'tell': False, 'gotta': False, 'come': False, 'ill': Fal

In [ ]:
#df_ = pd.DataFrame(columns=['class'])
classes= []
for test in testing_data_t:
  #print(test[0])
  #print(t_features," : ", classfier.classify(test[0]))
  classes.append(classfier.classify(test[0]))
  #dataframe = dataframe.append({'class':row['class'], 'tweet': ' '.join(clean_sentence)}, ignore_index = True)

In [ ]:
#df_ = pd.DataFrame(columns=['class'])
classes_test= []
for test in testing_data:
  #print(test[0])
  #print(t_features," : ", classfier.classify(test[0]))
  classes_test.append(classfier.classify(test))
  #dataframe = dataframe.append({'class':row['class'], 'tweet': ' '.join(clean_sentence)}, ignore_index = True)

In [ ]:
original = []
for test in testing_data_t:
  original.append(test[1])

In [ ]:
df_ = pd.DataFrame(data = classes, columns=['class_t'])
df_['class_test'] = classes_test
df_['real_class'] = original

In [ ]:
df_.head()

,class_t,class_test
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1


In [ ]:
raw_test['class']=df_['class_test']

In [ ]:
raw_test.to_excel('hate_speech_874384.xlsx', index = False)

In [ ]:
test_data['predicted'] = df_['class_test']
test_data.to_excel('hate_speech_comparison.xlsx', index = True)

In [ ]:
total = 0
count = 0
for index,row in test_data.iterrows():
  if row['class'] == row['predicted']:
    count+=1
  total+=1

accuracy = ((count/total)*100)
print(accuracy)

87.2100060520476


In [ ]:
test_data['predicted'].value_counts()

1    4074
2     734
0     149
Name: predicted, dtype: int64

In [ ]:
test_data['class'].value_counts()

1    3839
2     829
0     289
Name: class, dtype: int64

In [ ]:
for test in tests:
    t_features = {word: (word in word_tokenize(test.lower())) for word in features}
    print(t_features)

{'bitch': False, 'bitches': False, 'like': True, 'hoes': False, 'pussy': False, 'dont': False, 'hoe': False, 'ass': False, 'get': False, 'fuck': False, 'got': False, 'not': False, 'shit': False, 'aint': False, 'nigga': False, 'trash': False, 'lol': False, 'amp': False, 'know': False, 'niggas': False, 'cant': False, 'one': False, 'love': False, 'yall': False, 'fucking': False, 'thats': False, 'youre': False, 'want': False, 'bad': False, 'still': False, 'good': False, 'man': False, 'make': False, 'need': False, 'hate': False, 'say': False, 'see': False, 'look': False, 'time': False, 'think': False, 'back': False, 'really': True, 'even': False, 'never': False, 'people': False, 'white': False, 'real': False, 'faggot': False, 'would': False, 'girl': False, 'right': False, 'lmao': False, 'said': False, 'wanna': False, 'damn': False, 'bird': False, 'bout': False, 'let': False, 'little': False, 'dick': False, 'wit': False, 'stop': False, 'tell': False, 'talk': False, 'gonna': False, 'dat': Fal

In [ ]:
for test in testing_data:
  print(classfier.classify(test))
  break

In [ ]:
classfier.show_most_informative_features(15)

Most Informative Features
                     ass = True                0 : 2      =    128.0 : 1.0
                  nigger = True                0 : 2      =     96.3 : 1.0
                   bitch = True                1 : 2      =     87.0 : 1.0
                 yankees = True                2 : 1      =     78.1 : 1.0
                    oreo = True                2 : 1      =     63.4 : 1.0
                   jeter = True                2 : 1      =     62.8 : 1.0
                    ho&; = True                2 : 1      =     59.8 : 1.0
                niggers. = True                0 : 1      =     59.3 : 1.0
                  jihadi = True                2 : 1      =     57.9 : 1.0
                    gook = True                0 : 1      =     50.2 : 1.0
                 massage = True                2 : 1      =     47.5 : 1.0
                 charlie = True                2 : 1      =     45.8 : 1.0
                  yankee = True                2 : 1      =     41.4 : 1.0

In [ ]:
from sklearn.model_selection import train_test_split
X = dataframe.tweet.values
Y = dataframe.classes
Y = Y.astype('int')

x_train, x_test, y_train, y_test = train_test_split(X,Y)


In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(14869,)
(4957,)
(14869,)
(4957,)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
my_vectorizer = vectorizer.fit_transform(x_train)
vocab = vectorizer.vocabulary_

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(my_vectorizer,y_train)
my_test_data = vectorizer.transform(x_test)
my_pred = clf.predict(my_test_data)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
 from sklearn.naive_bayes import MultinomialNB
 nb_clf = MultinomialNB()
 nb_clf.fit(my_vectorizer,y_train)
 my_test_data = vectorizer.transform(x_test)
 my_pred = nb_clf.predict(my_test_data)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(my_pred, y_test)

0.7748638289287876

In [ ]:
clf.predict(vectorizer.transform(['I like this']))

array([1])

In [ ]:
df_pred = pd.DataFrame(data =my_pred)
df_pred.value_counts()

1    4946
2      10
0       1
dtype: int64

In [ ]:
dataframe['classes'].value_counts()

1    15350
2     3335
0     1141
Name: classes, dtype: int64